**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D, Reshape, BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action = 4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The function _act_ takes the current state *s* as input and returns a possible action. 

> In the train mode, we first random a seed between 0 and 1. If it is smaller than _epsilon_, we sample an act uniformly from the four choices; Otherwise, the use a deterministic strategy to choose the best action so far we learnt.

> In the val mode, we choose everytime the best action we learnt without any uncertainty.

> _epsilon_ plays a very important role in the function since it decides the balance between the exploration and the exploitation. If it is too small, there will be many actions which we never explore, which might leads us to learn all the time a bad strategy. However if it is too large, the model will be difficult to converge, which again leads to a bad model.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose - reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [26]:
class Environment(object):
    def __init__(self, grid_size = 10, max_time = 500, temperature = 0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        # board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the rat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0
        self.scale = 16
        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size, self.grid_size, 3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :]=256
        b[-2:, :, :]=0
        b[:, -2:, :]=0
        b[:2, :, :]=0
        b[:, :2, :]=0
        
        b = cv2.resize(b, None, fx = self.scale, fy = self.scale, interpolation = cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[:2, :] = -1
        self.position[:, :2] = -1
        self.position[:, -2:] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)), axis = 2)
        state = state[self.x - 2 : self.x + 3, self.y - 2 : self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size - 3, size = 1)[0]
        self.y = np.random.randint(3, self.grid_size - 3, size = 1)[0]


        bonus = 0.5 * np.random.binomial(1, self.temperature, size = self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size = self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))


        malus[bonus > 0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[:2, :]= -1
        self.position[:, :2] = -1
        self.position[:, -2:] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)), axis = 2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [27]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 200 # set small when debugging
epochs_test = 20 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The ```position``` keeps track of the invalid voxels, which are marked as -1, and the current position of the cat, which is marked as 1.

> The ```board``` keeps track of the voxels with bonus, which are marked as 0.5, and the voxels with malus, which are marked as -1. 

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    
    def __init__(self):
        super(RandomAgent, self).__init__()

    def learned_act(self, s):
        return np.random.randint(4)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent, env, epochs, prefix = ''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        # Intialize the game, flag and loss
        state = env.reset()
        game_over = False
        win, lose = 0, 0
        
        while not game_over:
            # sample an action
            action = agent.act(state, train = False)
            # perform an action
            prev_state = state
            state, reward, game_over = env.act(action)
            
            # update loss
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward
        
        # Save as a mp4
        env.draw(prefix + str(e))

        # Update stats
        score = score + win - lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score / (1 + e)))
        
    print('Final score: ' + str(score / epochs))

In [9]:
# Initialize the game
env = Environment(grid_size = size, max_time = T,temperature = temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent, env, epochs_test, prefix = 'models/random')

HTML(display_videos('models/random0.mp4'))

Win/lose count 11.0/20.0. Average score (-9.0)
Win/lose count 14.0/15.0. Average score (-5.0)
Final score: -5.0


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

> According to the definition:
 \begin{align*}
    Q^\pi (s,a) &= E_{p^\pi} [\sum_{t=0}^\infty \gamma^t r(s_t, a_t)|s_0 = s, a_0 = a] \\
    &= E_{p^\pi} [r_0 + \gamma \sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1})|s_0 = s, a_0 = a]
  \end{align*}
  
> The first term can be written as \begin{align*}
    E_{p^\pi} [r_0|s_0 = s, a_0 = a] &= \sum_{s'} p(s'|s, a) r(s, a) \\
  \end{align*}
  
> The second term can be written as 

>\begin{align*}
    E_{p^\pi} [\gamma \sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1})|s_0 = s, a_0 = a] &= \sum_{s'} p(s'|s, a) \gamma E_{p^\pi} [\sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1}) | s_1 = s']\\
  \end{align*}
  
> Thus the sum is 
  \begin{align*}
    Q^\pi (s,a) &=  \sum_{s'} p(s'|s, a) \Big (r(s, a) + \gamma \sum_{a'} E_{p^\pi} [\sum_{t=0}^\infty \gamma^t r(s_{t+1}, a_{t+1}) | s_1 = s', a_1 = a'] \Big ) \\
    &= \sum_{s'} p(s'|s, a) \Big (r(s, a) + \gamma Q^\pi (s', a') \Big ) \\
    &= E_{(s',a') \thicksim p(\cdot|s, a)} [r(s,a) + \gamma Q^\pi (s', a')]
  \end{align*}
  
> Given that the optimal policy $\pi^* = \mbox{argmax}_{a} = Q^* (s,a)$ exists, we have

>  \begin{align*}
    Q^*(s,a) &= \max_{\pi}Q^\pi(s,a) = Q^{\pi^*}(s,a) \\
    &= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]
  \end{align*}
  
> What we want to minimize is the difference between the Q-learning target $Q^*(s,a, \theta)$ and the Q-learning prediction $Q(s,a, \theta)$, which can be formulated into the following least-square problem:

> \begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory = 100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory.pop(0)
            self.memory.append(m)
            
    def random_access(self):
        
        index = np.random.randint(len(self.memory))
        
        return self.memory[index]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent, env, epoch, prefix = ''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win - lose))
        agent.save(name_weights = prefix + 'model.h5', name_model = prefix + 'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon = 0.1, memory_size = 100, batch_size = 16, n_state = 2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        
        pred_q = self.model.predict(s[np.newaxis, :, :, :])
        return np.argmax(pred_q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i, :, :] = s
            target_q[i] = self.model.predict(s[np.newaxis, :, :, :])[0]
 
            if game_over_:
                target_q[i][a] = r
            else:
                n_q = max(self.model.predict(n_s[np.newaxis, :, :, :])[0])
                target_q[i][a] = r + n_q * self.discount
                
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr = 0.1, **kwargs):
        super(DQN_FC, self).__init__( *args, **kwargs)
        
        # NN Model
        model = Sequential()
        model.add(Reshape((5 * 5 * self.n_state,)))
        model.add(Dense(25, input_dim = 5 * 5 * self.n_state, activation = 'relu'))
        model.add(Dense(15, activation = 'relu'))
        model.add(Dense(4, activation = 'linear'))

        model.compile(sgd(lr = lr, decay = 1e-4, momentum = 0.0), "mse")
        self.model = model

In [21]:
env = Environment(grid_size = size, max_time = T, temperature = 0.3)
agent = DQN_FC(size, lr = 0.01, epsilon = 0.1, memory_size = 2000, batch_size = 64)
train(agent, env, epochs_train, prefix = 'models/fc_train')
HTML(display_videos('models/fc_train10.mp4'))

Epoch 000/200 | Loss 0.0446 | Win/lose count 5.0/7.0 (-2.0)
Epoch 001/200 | Loss 0.0590 | Win/lose count 2.5/8.0 (-5.5)
Epoch 002/200 | Loss 0.0620 | Win/lose count 3.5/6.0 (-2.5)
Epoch 003/200 | Loss 0.0500 | Win/lose count 5.0/5.0 (0.0)
Epoch 004/200 | Loss 0.0428 | Win/lose count 2.0/2.0 (0.0)
Epoch 005/200 | Loss 0.0457 | Win/lose count 6.0/6.0 (0.0)
Epoch 006/200 | Loss 0.0424 | Win/lose count 2.0/5.0 (-3.0)
Epoch 007/200 | Loss 0.0411 | Win/lose count 0.5/2.0 (-1.5)
Epoch 008/200 | Loss 0.0330 | Win/lose count 1.5/0 (1.5)
Epoch 009/200 | Loss 0.0349 | Win/lose count 1.5/4.0 (-2.5)
Epoch 010/200 | Loss 0.0267 | Win/lose count 4.5/2.0 (2.5)
Epoch 011/200 | Loss 0.0199 | Win/lose count 3.5/7.0 (-3.5)
Epoch 012/200 | Loss 0.0303 | Win/lose count 4.0/3.0 (1.0)
Epoch 013/200 | Loss 0.0258 | Win/lose count 2.5/2.0 (0.5)
Epoch 014/200 | Loss 0.0177 | Win/lose count 3.5/2.0 (1.5)
Epoch 015/200 | Loss 0.0177 | Win/lose count 3.5/6.0 (-2.5)
Epoch 016/200 | Loss 0.0313 | Win/lose count 2.0/7

Epoch 139/200 | Loss 0.0365 | Win/lose count 7.0/6.0 (1.0)
Epoch 140/200 | Loss 0.0376 | Win/lose count 4.5/2.0 (2.5)
Epoch 141/200 | Loss 0.0422 | Win/lose count 2.0/4.0 (-2.0)
Epoch 142/200 | Loss 0.0507 | Win/lose count 7.5/5.0 (2.5)
Epoch 143/200 | Loss 0.0320 | Win/lose count 2.0/1.0 (1.0)
Epoch 144/200 | Loss 0.0346 | Win/lose count 1.0/5.0 (-4.0)
Epoch 145/200 | Loss 0.0302 | Win/lose count 6.5/2.0 (4.5)
Epoch 146/200 | Loss 0.0330 | Win/lose count 5.0/9.0 (-4.0)
Epoch 147/200 | Loss 0.0301 | Win/lose count 2.5/3.0 (-0.5)
Epoch 148/200 | Loss 0.0429 | Win/lose count 5.5/2.0 (3.5)
Epoch 149/200 | Loss 0.0419 | Win/lose count 4.5/0 (4.5)
Epoch 150/200 | Loss 0.0341 | Win/lose count 5.0/7.0 (-2.0)
Epoch 151/200 | Loss 0.0372 | Win/lose count 3.5/2.0 (1.5)
Epoch 152/200 | Loss 0.0474 | Win/lose count 4.5/6.0 (-1.5)
Epoch 153/200 | Loss 0.0458 | Win/lose count 1.5/1.0 (0.5)
Epoch 154/200 | Loss 0.0472 | Win/lose count 6.5/3.0 (3.5)
Epoch 155/200 | Loss 0.0389 | Win/lose count 5.0/8.0

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args, lr = 0.1, **kwargs):
        super(DQN_CNN, self).__init__(*args, **kwargs)
        
        model = Sequential()
        model.add(Conv2D(32, 2, padding = 'same', input_shape = (5, 5, self.n_state)))
        model.add(Activation('relu'))
        model.add(Conv2D(64, 2, padding = 'same'))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(512, activation = 'relu'))
        model.add(Dense(128, activation = 'relu'))
        model.add(Dense(4, activation = 'linear'))

        model.compile(sgd(lr = lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [22]:
env = Environment(grid_size = size, max_time = T, temperature = 0.3)
agent = DQN_CNN(size, lr = 0.01, epsilon = 0.1, memory_size = 2000, batch_size = 64)
train(agent, env, epochs_train, prefix = 'models/cnn_train')
HTML(display_videos('models/cnn_train10.mp4'))

Epoch 000/200 | Loss 0.0114 | Win/lose count 3.0/3.0 (0.0)
Epoch 001/200 | Loss 0.0138 | Win/lose count 4.0/5.0 (-1.0)
Epoch 002/200 | Loss 0.0195 | Win/lose count 1.5/4.0 (-2.5)
Epoch 003/200 | Loss 0.0124 | Win/lose count 1.5/0 (1.5)
Epoch 004/200 | Loss 0.0127 | Win/lose count 1.0/3.0 (-2.0)
Epoch 005/200 | Loss 0.0124 | Win/lose count 3.5/5.0 (-1.5)
Epoch 006/200 | Loss 0.0099 | Win/lose count 2.0/1.0 (1.0)
Epoch 007/200 | Loss 0.0158 | Win/lose count 5.5/3.0 (2.5)
Epoch 008/200 | Loss 0.0052 | Win/lose count 5.0/5.0 (0.0)
Epoch 009/200 | Loss 0.0097 | Win/lose count 4.0/6.0 (-2.0)
Epoch 010/200 | Loss 0.0173 | Win/lose count 2.0/3.0 (-1.0)
Epoch 011/200 | Loss 0.0089 | Win/lose count 0.5/1.0 (-0.5)
Epoch 012/200 | Loss 0.0044 | Win/lose count 2.5/3.0 (-0.5)
Epoch 013/200 | Loss 0.0136 | Win/lose count 3.5/4.0 (-0.5)
Epoch 014/200 | Loss 0.0044 | Win/lose count 2.5/1.0 (1.5)
Epoch 015/200 | Loss 0.0078 | Win/lose count 3.0/5.0 (-2.0)
Epoch 016/200 | Loss 0.0086 | Win/lose count 4.0

Epoch 139/200 | Loss 0.6327 | Win/lose count 6.5/0 (6.5)
Epoch 140/200 | Loss 0.6260 | Win/lose count 4.5/2.0 (2.5)
Epoch 141/200 | Loss 0.6250 | Win/lose count 4.0/1.0 (3.0)
Epoch 142/200 | Loss 0.6424 | Win/lose count 7.5/3.0 (4.5)
Epoch 143/200 | Loss 0.6744 | Win/lose count 5.5/0 (5.5)
Epoch 144/200 | Loss 0.6860 | Win/lose count 9.5/1.0 (8.5)
Epoch 145/200 | Loss 0.7091 | Win/lose count 5.5/3.0 (2.5)
Epoch 146/200 | Loss 0.7426 | Win/lose count 10.0/3.0 (7.0)
Epoch 147/200 | Loss 0.8063 | Win/lose count 6.0/2.0 (4.0)
Epoch 148/200 | Loss 0.8225 | Win/lose count 10.0/1.0 (9.0)
Epoch 149/200 | Loss 0.8358 | Win/lose count 1.5/1.0 (0.5)
Epoch 150/200 | Loss 0.9160 | Win/lose count 13.5/3.0 (10.5)
Epoch 151/200 | Loss 1.0208 | Win/lose count 6.0/0 (6.0)
Epoch 152/200 | Loss 1.0658 | Win/lose count 6.0/4.0 (2.0)
Epoch 153/200 | Loss 1.1832 | Win/lose count 3.0/5.0 (-2.0)
Epoch 154/200 | Loss 1.1344 | Win/lose count 6.0/4.0 (2.0)
Epoch 155/200 | Loss 1.0667 | Win/lose count 9.5/2.0 (7.5

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [25]:
env = Environment(grid_size = size, max_time = 300,temperature = 0.3)
agent_cnn = DQN_CNN(size, lr = .1, epsilon = 0.01, memory_size = 2000, batch_size = 32)
agent_cnn.load(name_weights = 'models/cnn_trainmodel.h5', name_model = 'models/cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr = .1, epsilon = 0.01, memory_size = 2000, batch_size = 32)
agent_fc.load(name_weights = 'models/fc_trainmodel.h5', name_model = 'models/fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn, env, epochs_test, prefix = 'models/cnn_test')
print('Test of the FC')
test(agent_fc, env, epochs_test, prefix = 'models/fc_test')

Test of the CNN
Win/lose count 4.5/2.0. Average score (2.5)
Win/lose count 4.0/0. Average score (3.25)
Win/lose count 3.0/1.0. Average score (2.8333333333333335)
Win/lose count 0.5/0. Average score (2.25)
Win/lose count 1.0/0. Average score (2.0)
Win/lose count 0.5/0. Average score (1.75)
Win/lose count 0/0. Average score (1.5)
Win/lose count 1.0/1.0. Average score (1.3125)
Win/lose count 2.0/1.0. Average score (1.2777777777777777)
Win/lose count 1.0/0. Average score (1.25)
Win/lose count 0.5/0. Average score (1.1818181818181819)
Win/lose count 0.5/1.0. Average score (1.0416666666666667)
Win/lose count 0/0. Average score (0.9615384615384616)
Win/lose count 1.5/0. Average score (1.0)
Win/lose count 1.0/0. Average score (1.0)
Final score: 1.0
Test of the FC
Win/lose count 0.5/0. Average score (0.5)
Win/lose count 0/2.0. Average score (-0.75)
Win/lose count 0/1.0. Average score (-0.8333333333333334)
Win/lose count 0/0. Average score (-0.625)
Win/lose count 1.5/1.0. Average score (-0.4)
Wi

In [95]:
HTML(display_videos('cnn_test10.mp4'))

In [96]:
HTML(display_videos('fc_test10.mp4'))

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [16]:
def train_explore(agent, env, epoch, prefix = ''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        agent.set_epsilon(max(0.1, np.exp(-0.5 * np.sqrt(e))))
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train = True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win - lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win - lose))
        agent.save(name_weights = prefix + 'model.h5',name_model = prefix + 'model.json')

In [30]:
class EnvironmentExploring(Environment):
    
    def __init__(self, grid_size = 10, max_time = 500, temperature = 0.1):
        
        super(EnvironmentExploring, self).__init__(grid_size, max_time, temperature)

        # board on which one plays
        self.malus_position = np.zeros((grid_size, grid_size))

    def act(self, action, train = False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[:, -2:] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size - 3:
                self.x = self.x - 1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x + 1
            else:
                self.x = self.x - 1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = -self.malus_position[self.x, self.y] if train else 0
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        self.malus_position[self.x, self.y] = 0.1
        game_over = self.t > self.max_time
        
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)), axis = 2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size - 3, size = 1)[0]
        self.y = np.random.randint(3, self.grid_size - 3, size = 1)[0]

        bonus = 0.5 * np.random.binomial(1, self.temperature, size = self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size = self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))

        malus[bonus > 0] = 0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0 : 2, :]= -1
        self.position[:, 0 : 2] = -1
        self.position[:, -2:] = -1
        self.position[-2:, :] = -1
        self.board[self.x, self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        self.malus_position[self.x, self.y] = 0.1

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                                self.board.reshape(self.grid_size, self.grid_size, 1),
                                self.position.reshape(self.grid_size, self.grid_size, 1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [23]:
# Training
env = EnvironmentExploring(grid_size = size, max_time = T, temperature = 0.3)
agent = DQN_CNN(size, lr = 0.01, epsilon = 0.1, memory_size = 2000, batch_size = 32,n_state = 3)
train_explore(agent, env, epochs_train, prefix = 'models/cnn_train_explore')
HTML(display_videos('models/cnn_train_explore10.mp4'))

Epoch 000/200 | Loss 0.0115 | Win/lose count 10.0/22.80000000000004 (-12.80000000000004)
Epoch 001/200 | Loss 0.0086 | Win/lose count 10.5/22.80000000000007 (-12.300000000000072)
Epoch 002/200 | Loss 0.0032 | Win/lose count 9.5/27.80000000000009 (-18.30000000000009)
Epoch 003/200 | Loss 0.0243 | Win/lose count 3.0/27.50000000000012 (-24.50000000000012)
Epoch 004/200 | Loss 0.0165 | Win/lose count 14.0/28.300000000000054 (-14.300000000000054)
Epoch 005/200 | Loss 0.0186 | Win/lose count 8.5/24.80000000000005 (-16.30000000000005)
Epoch 006/200 | Loss 0.0323 | Win/lose count 5.5/23.300000000000036 (-17.800000000000036)
Epoch 007/200 | Loss 0.0243 | Win/lose count 6.0/21.100000000000023 (-15.100000000000023)
Epoch 008/200 | Loss 0.0313 | Win/lose count 4.0/21.500000000000043 (-17.500000000000043)
Epoch 009/200 | Loss 0.0442 | Win/lose count 8.5/22.600000000000062 (-14.100000000000062)
Epoch 010/200 | Loss 0.0728 | Win/lose count 6.0/20.40000000000002 (-14.40000000000002)
Epoch 011/200 | Lo

Epoch 093/200 | Loss 1.3656 | Win/lose count 20.5/14.099999999999971 (6.400000000000029)
Epoch 094/200 | Loss 1.3166 | Win/lose count 25.5/11.29999999999998 (14.20000000000002)
Epoch 095/200 | Loss 1.2383 | Win/lose count 18.5/13.999999999999966 (4.500000000000034)
Epoch 096/200 | Loss 1.2763 | Win/lose count 20.5/12.499999999999972 (8.000000000000028)
Epoch 097/200 | Loss 1.2483 | Win/lose count 24.5/8.899999999999984 (15.600000000000016)
Epoch 098/200 | Loss 1.2958 | Win/lose count 17.5/16.19999999999997 (1.3000000000000291)
Epoch 099/200 | Loss 1.2182 | Win/lose count 17.0/12.69999999999997 (4.300000000000029)
Epoch 100/200 | Loss 1.1739 | Win/lose count 19.0/18.099999999999998 (0.9000000000000021)
Epoch 101/200 | Loss 1.1527 | Win/lose count 28.0/15.099999999999973 (12.900000000000027)
Epoch 102/200 | Loss 1.0383 | Win/lose count 20.0/16.899999999999977 (3.1000000000000227)
Epoch 103/200 | Loss 1.0454 | Win/lose count 11.0/14.899999999999963 (-3.899999999999963)
Epoch 104/200 | Los

Epoch 185/200 | Loss 1.1787 | Win/lose count 23.5/11.29999999999998 (12.20000000000002)
Epoch 186/200 | Loss 0.9900 | Win/lose count 19.5/15.499999999999972 (4.000000000000028)
Epoch 187/200 | Loss 1.1840 | Win/lose count 25.0/10.499999999999979 (14.500000000000021)
Epoch 188/200 | Loss 0.9824 | Win/lose count 23.5/13.399999999999975 (10.100000000000025)
Epoch 189/200 | Loss 1.0846 | Win/lose count 24.5/12.399999999999983 (12.100000000000017)
Epoch 190/200 | Loss 1.1379 | Win/lose count 24.5/11.899999999999974 (12.600000000000026)
Epoch 191/200 | Loss 1.2834 | Win/lose count 28.0/10.299999999999988 (17.70000000000001)
Epoch 192/200 | Loss 1.1585 | Win/lose count 31.0/14.499999999999975 (16.500000000000025)
Epoch 193/200 | Loss 1.0734 | Win/lose count 27.0/9.89999999999998 (17.10000000000002)
Epoch 194/200 | Loss 1.1024 | Win/lose count 23.5/14.199999999999969 (9.300000000000031)
Epoch 195/200 | Loss 1.1871 | Win/lose count 28.0/12.89999999999998 (15.10000000000002)
Epoch 196/200 | Loss

In [38]:
# Evaluation
agent.load(name_weights = 'models/cnn_train_exploremodel.h5', name_model = 'models/cnn_train_exploremodel.json')
test(agent, env, epochs_test, prefix = 'models/cnn_test_explore')
HTML(display_videos('models/cnn_test_explore10.mp4'))

Win/lose count 26.0/0. Average score (26.0)
Win/lose count 21.5/1.0. Average score (23.25)
Win/lose count 6.0/0. Average score (17.5)
Win/lose count 18.0/1.0. Average score (17.375)
Win/lose count 21.0/0. Average score (18.1)
Win/lose count 19.0/0. Average score (18.25)
Win/lose count 17.0/2.0. Average score (17.785714285714285)
Win/lose count 25.0/2.0. Average score (18.4375)
Win/lose count 17.5/0. Average score (18.333333333333332)
Win/lose count 24.0/0. Average score (18.9)
Win/lose count 12.0/1.0. Average score (18.181818181818183)
Win/lose count 23.0/0. Average score (18.583333333333332)
Win/lose count 16.5/1.0. Average score (18.346153846153847)
Win/lose count 7.0/0. Average score (17.535714285714285)
Win/lose count 18.0/1.0. Average score (17.5)
Win/lose count 17.5/0. Average score (17.5)
Win/lose count 13.5/0. Average score (17.264705882352942)
Win/lose count 5.5/0. Average score (16.61111111111111)
Win/lose count 17.5/0. Average score (16.657894736842106)
Win/lose count 21.5/0

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***